# Test Endpoint Machine Learning

In [14]:
#import library
import requests
import pandas as pd
import tensorflow as tf
import json
import base64

In [15]:
data = pd.read_csv('data/diabetes_prediction_dataset.csv')
inputs = data.iloc[51, :8].to_dict()
inputs

{'gender': 'Female',
 'age': 26.0,
 'hypertension': 0,
 'heart_disease': 0,
 'smoking_history': 'No Info',
 'bmi': 27.32,
 'HbA1c_level': 4.0,
 'blood_glucose_level': 200}

In [16]:
data.iloc[51, :9]

gender                  Female
age                       26.0
hypertension                 0
heart_disease                0
smoking_history        No Info
bmi                      27.32
HbA1c_level                4.0
blood_glucose_level        200
diabetes                     0
Name: 51, dtype: object

In [17]:
def prepare_json(inputs: dict):
    feature_spec = dict()

    for keys, values in inputs.items():
        if isinstance(values, float):
            feature_spec[keys] = tf.train.Feature(float_list=tf.train.FloatList(value=[values]))
        elif isinstance(values, int):
            feature_spec[keys] = tf.train.Feature(int64_list=tf.train.Int64List(value=[values]))
        elif isinstance(values, str):
            feature_spec[keys] = tf.train.Feature(bytes_list=tf.train.BytesList(value=[values.encode()]))

    example = tf.train.Example(
        features=tf.train.Features(feature=feature_spec)
    ).SerializeToString()

    result = [
        {
            "examples": {
                "b64": base64.b64encode(example).decode()
            }
        }
    ]

    return json.dumps({
        "signature_name": "serving_default",
        "instances": result,
    })

json_data = prepare_json(inputs)

endpoint = "https://diabetes-classification-production-7489.up.railway.app/v1/models/diabetes-model:predict"
response = requests.post(endpoint, data=json_data)

prediction = response.json().get("predictions")
if prediction:
    prediction_value = prediction[0][0]
    result = "No diabetes" if prediction_value < 0.5 else "Diabetes"
else:
    result = "Error: No predictions found in response."

print(result)

No diabetes


In [18]:
import requests
from pprint import PrettyPrinter
 
pp = PrettyPrinter()
pp.pprint(requests.get("https://diabetes-classification-production-7489.up.railway.app/v1/models/diabetes-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1704442580'}]}
